Digital filter initialization is one way around the mass imbalance issues we have with the NG-Aqua input data. 

First, we should see if the transient problems in the boundary layer disappear at the end of 3-6 hour integration.

In [ ]:
import holoviews as hv
hv.extension('bokeh')
%opts Image[width=500, height=300, colorbar=True](cmap='magma') 
%opts QuadMesh[width=300,  height=150, colorbar=True](cmap='viridis')
%opts Curve[width=500, height=int(500/1.61)]

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from sam.case import get_ngqaua_ic, InitialConditionCase
import xarray as xr
from os.path import join
import numpy as np
from uwnet.thermo import layer_mass


def process_ic(ic, **kwargs):

    # open initial conditions
    case = InitialConditionCase(ic=ic)

    # configure the sam run
    case.prm['parameters'].update({
        'dosgs': False,
        'dowally': True,
        'nstop': 180,
        'nsave3d': 30,
        'nsave2d': 30,
        'nsave2dstart': 0 ,
        'nstat': 15,
        'nstatfrq': 1,
        'dt': 120,
    })
    
    case.prm['parameters'].update(kwargs)

    case.save()

    # Run the model
    print(case.path)
    !cd {case.path} &&  bash -c './run.sh > out  2> err'
    # !cd {case.path} &&  bash -c './run.sh'

    # open the 3d outputs
    data_3d = xr.open_mfdataset(join(case.path, 'OUT_3D', '*.nc'), concat_dim='time').load()
    assert len(data_3d.time) > 1
    
    data_2d = xr.open_mfdataset(join(case.path, 'OUT_2D', '*.nc'), concat_dim='time').load()
    
    stat = xr.open_mfdataset(join(case.path, 'OUT_STAT', '*.nc'), concat_dim='time').load()
    
    # clean up the working directory
    !rm -rf {case.path}
    
    return data_3d, data_2d, stat


def plot_diffs(sam_processed):
    dsam = sam_processed.diff('time')

    # get the surface level
    dsam_sfc = dsam.isel(z=0)
    
    lay = hv.Dataset(dsam_sfc.TABS).to.image(["x", "y"], label="T(i+1) - T(i)") \
     + hv.Dataset(dsam_sfc.QT).to.image(["x", "y"], label="QT(i+1) - Qt(i)")

    return lay.cols(1)

Load the initial condition and run for 6 hours

In [ ]:
NGROOT = "/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/"
STATFILE = join(NGROOT, 'stat.nc')

stat = xr.open_mfdataset(STATFILE)
rho = stat.RHO[0]

# layer mass
dm = layer_mass(rho)

# get the initial condition at first time step
# the winds are staggered
ic = get_ngqaua_ic(NGROOT, 0)

# Process this initial conditions using SAM
data_3d_cori, data_2d, stat = process_ic(ic, dt=120, nstop=360, nsave3d=30, nsave2d=30, nstat=15)

In [ ]:
# Take the difference in time.
dsam = data_3d_cori.diff('time')

# get the surface level
dsam_sfc = dsam.isel(z=0)

In [ ]:
%%opts Image{+framewise}

lay = hv.Dataset(dsam_sfc.TABS).to.image(["x", "y"], label="T(i+1) - T(i)") \
 + hv.Dataset(dsam_sfc.QT).to.image(["x", "y"], label="QT(i+1) - Qt(i)")

lay.cols(1)

In [ ]:
plt.pcolor(data_3d_cori.W.mean(['x', 'y']).T)

This shows that divergence is accumulating in the simulation.Chris again thinks this is because the model does not actually use a projection method. Therefore, using the wrong boundary conditions will allow divergence to accumulate in time. Does this actually happen?

Does the problem get better if I turn off the coriolis force?

In [ ]:
# Process this initial conditions using SAM
sam_processed = process_ic(ic, dt=120, nstop=360, nsave3d=60, nsave2d=60, nstat=30, docoriolis=False)

In [ ]:
plot_diffs(sam_processed[0])

In [ ]:
div = sam_processed[0].W.mean(['x', 'y'])

In [ ]:
plt.pcolor(div.values.T, cmap='RdBu_r')
plt.colorbar()

We can see that the first time step removes all the divergence, then there is a peak of divergence, which is ultimately damped to nothing.

# Pressure gradient at boundary

From the $y$ momentum equation we known that the balance $ f u = - p_x/\rho_0 $ should be true on the boundary. How well is this satisfied?

In [ ]:
from uwnet.thermo import coriolis_ngaqua

# open data
ng_3d = xr.open_mfdataset(join(NGROOT, 'coarse', '3d', 'all.nc')).isel(time=0)
rho = xr.open_mfdataset(join(NGROOT, 'stat.nc')).RHO[0]

# compute Py an
py = -(ng_3d.PP.isel(y=1) - ng_3d.PP.isel(y=0))/160e3/rho
fcori = coriolis_ngaqua(ng_3d.y)
fu = ng_3d.U.isel(y=0) * fcori[0]

lay = hv.Dataset(py, vdims=['py'], label='-P_y/rho').to.curve("x")\
*hv.Dataset(fu, vdims=['py'], label='f * U').to.curve("x")

Note that the pressure gradient I compute here is located between the first and second pressure points, which is $1.5 \cdot 160$ km away from the boundary.

In [ ]:
%%output size=150
%%opts Curve[width=300, height=161]
lay[::5].layout().cols(2)

We can see that the pressure gradient near the boundary is furthest from geostrophic balance at the lower heights. Maybe that is why the signal is most degraded there.

Here is the zonal velocity near the southern boundary:

In [ ]:
fu.plot()

And the meridional gradient of pressure. I might expect this gradient to be closer to zero since it is only one grid point away from the boundary.

In [ ]:
py.plot()

# Conclusions

These results do seem to indicate that Chris could be right that the y-boundary conditions for the pressure solver are causing the zonal striping problems. 

Here is the code for computing the pressure right hand side:

```fortran
do k=1,nzm
 kc=k+1 
 rdz=1./(adz(k)*dz)
 rup = rhow(kc)/rho(k)*rdz
 rdn = rhow(k)/rho(k)*rdz
 do j=1,ny
  jc=j+1 
  do i=1,nx
   ic=i+1
   p(i,j,k)=(rdx*(u(ic,j,k)-u(i,j,k))+ &
             rdy*(v(i,jc,k)-v(i,j,k))+ &
             (w(i,j,kc)*rup-w(i,j,k)*rdn) )*dta + &	
            (rdx*(dudt(ic,j,k,na)-dudt(i,j,k,na))+ &
             rdy*(dvdt(i,jc,k,na)-dvdt(i,j,k,na))+ &
             (dwdt(i,j,kc,na)*rup-dwdt(i,j,k,na)*rdn) ) + &
       btat*(rdx*(dudt(ic,j,k,nb)-dudt(i,j,k,nb))+ &
             rdy*(dvdt(i,jc,k,nb)-dvdt(i,j,k,nb))+ &
             (dwdt(i,j,kc,nb)*rup-dwdt(i,j,k,nb)*rdn) ) + &
       ctat*(rdx*(dudt(ic,j,k,nc)-dudt(i,j,k,nc))+ &
             rdy*(dvdt(i,jc,k,nc)-dvdt(i,j,k,nc))+ &
             (dwdt(i,j,kc,nc)*rup-dwdt(i,j,k,nc)*rdn) )
   p(i,j,k)=p(i,j,k)*rho(k)
  end do
 end do
end do

```

This appears include a contribution from the divergence (the first three lines), and then a section which includes the tendency information. So when `dudt` and the other tendencies are zero, the method is the same as a projection method. This explains why the first snapshot of these simulation has no divergence. Therefore, why not just remove this extra terms? What benefit do they provide? Higher order of accuracy?

## How to fix this?

1. Fix the boundary condition on the north south boundaries, and/or
2. Damp the winds towards near the boundaries. This should make $u=0$ there, so then the proper boundary condition would satisfied.